In [2]:
import io
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
#from sympy.solvers import solve
%matplotlib inline
filename = "mvar-set2.dat.txt"
X = []
Y = []
for line in open(filename):
    li=line.strip()
    if not li.startswith("#"):
        var = line.split()
        Y.append([float(var[-1])])
        X.append([float(i) for i in var[0:len(var)-1]])

In [3]:
'''creating the higher dimensions'''
poly = PolynomialFeatures(2)
Zhd2 = poly.fit_transform(X)

In [4]:
poly = PolynomialFeatures(3)
Zhd3 = poly.fit_transform(X)

In [5]:
poly = PolynomialFeatures(4)
Zhd4 = poly.fit_transform(X)

In [6]:
theta = np.dot(np.linalg.pinv(np.array(X)),np.array(Y))
theta_hd2 = np.dot(np.linalg.pinv(np.array(Zhd2)),np.array(Y))
theta_hd3 = np.dot(np.linalg.pinv(np.array(Zhd3)),np.array(Y))
theta_hd4 = np.dot(np.linalg.pinv(np.array(Zhd4)),np.array(Y))

In [7]:
print theta,"\n"
print theta_hd2,"\n"
print theta_hd3,"\n"
print theta_hd4,"\n"

[[ 0.06458313]
 [-0.00060792]] 

[[  4.27124879e-04]
 [  6.45831279e-02]
 [ -6.07924252e-04]
 [  2.05812667e-05]
 [ -1.07308260e-03]
 [  3.14956092e-04]] 

[[  4.27124879e-04]
 [  2.74640229e-01]
 [ -4.46822095e-04]
 [  2.05812667e-05]
 [ -1.07308260e-03]
 [  3.14956092e-04]
 [ -6.29961462e-02]
 [ -2.02197567e-04]
 [ -3.80321131e-02]
 [  4.78640087e-05]] 

[[ -3.05395075e-03]
 [  2.74640229e-01]
 [ -4.46822095e-04]
 [  5.79957163e-03]
 [  3.89628164e-03]
 [  2.21490074e-03]
 [ -6.29961462e-02]
 [ -2.02197567e-04]
 [ -3.80321131e-02]
 [  4.78640087e-05]
 [ -1.47536279e-03]
 [ -8.76002123e-04]
 [ -3.75542840e-04]
 [ -1.11442954e-03]
 [ -3.86890751e-04]] 



In [8]:
"cross validation procedure for 2-dimensional space"
zlen = len(Zhd2) #size of data given.
k_folds = 10 #number of folds to be used.
tmp_list = [] #This list will contain the floor values obtained.
indices = [] #Using the floor values we are going to determine the indices that are going to be used for cross validation.
for i in range(k_folds):
    floor = zlen//k_folds
    tmp_list.append(floor)
ini =0 #initial value
for i in tmp_list:
    fin = ini + i #final value, final value will be used as intial value for the successive record.
    indices.append((ini,fin))
    ini = fin
#print indices

Zt = np.ndarray([]) #train
Ztst = [] #test
Yt = np.ndarray([])
Ytst = []
theta_cv = {}
Y_train_dict = {}
Z_train_dict = {}
Y_dict = {} # Contains Ytst in the form of a dictionary, for being able to handle it better with Keys.
Z_dict ={}  #Same as Y_dict{}
key =1
for i in indices:
    Ztst = Zhd2[i[0]:i[1]]
    Z_dict[key] = Ztst
    Ytst = Y[i[0]:i[1]]
    Y_dict[key] = Ytst
    for j in indices:
            if i != j:
                Zt = Zt + Zhd2[j[0]:j[1]]
                Z_train_dict[key] = Zt
                Yt = Yt + Y[j[0]:j[1]]
                Y_train_dict[key] = Yt
    theta_cv[key] = np.linalg.solve((np.dot(np.array(Zt).transpose(),np.array(Zt))),
                           (np.dot(np.array(Zt).transpose(),np.array(Yt))))
    key = key +1
    
#Predicting the values. Comparing the values obtained using theta_cv vs y_dict
predicthd2 ={}
for i in theta_cv:
    t_list = []
    for j in Z_dict[i]:
        t_list.append(np.dot(np.array(theta_cv[i]).transpose(),np.array(j)))
    predicthd2[i] = t_list
    
predict_trainhd2 = {}
for i in theta_cv:
    t_list = []
    for j in Z_train_dict[i]:
        t_list.append(np.dot(np.array(theta_cv[i]).transpose(),np.array(j)))
    predict_trainhd2[i] = t_list
    
MSE_testhd2 = {}
for i in Y_dict:
    sumd = 0
    for m,n in zip(predicthd2[i],Y_dict[i]):
        sumd = sumd + (m[0]-n[0])**2
    MSE_testhd2[i] = sumd/len(predicthd2[i])
    
MSE_trainhd2 = {}
for i in Y_dict:
    sumd = 0
    for m,n in zip(predict_trainhd2[i],Y_train_dict[i]):
        sumd = sumd + (m[0]-n[0])**2
    MSE_trainhd2[i] = sumd/len(predict_trainhd2[i])

In [9]:
print MSE_testhd2,'\n'
print MSE_trainhd2,'\n'
print np.mean(MSE_testhd2.values()),'\n'
print np.mean(MSE_trainhd2.values())

{1: 0.019767046272096085, 2: 0.025101177766287477, 3: 0.021129631070282876, 4: 0.021753408612822791, 5: 0.018987651166274003, 6: 0.018997182705493321, 7: 0.019220902593277436, 8: 0.020727977027799507, 9: 0.019665144038266243, 10: 0.018297028963093872} 

{1: 0.16579318246104327, 2: 0.60376374692747314, 3: 1.3394607329002028, 4: 2.3365012455662884, 5: 3.659466076450788, 6: 5.2574372479437423, 7: 7.1218295765430266, 8: 9.3072918582578765, 9: 11.71105157201975, 10: 14.523349476629724} 

0.0203647150216 

5.60259447157


In [10]:
"cross validation procedure for 3-dimensional space"
zlen1 = len(Zhd3) #size of data given.
k_folds = 10 #number of folds to be used.
tmp_list = [] #This list will contain the floor values obtained.
indices = [] #Using the floor values we are going to determine the indices that are going to be used for cross validation.
for i in range(k_folds):
    floor = zlen1//k_folds
    tmp_list.append(floor)
ini =0 #initial value
for i in tmp_list:
    fin = ini + i #final value, final value will be used as intial value for the successive record.
    indices.append((ini,fin))
    ini = fin
#print indices

Zt = np.ndarray([]) #train
Ztst = [] #test
Yt = np.ndarray([])
Ytst = []
theta_cv = {}
Y_train_dict = {}
Z_train_dict = {}
Y_dict = {} # Contains Ytst in the form of a dictionary, for being able to handle it better with Keys.
Z_dict ={}  #Same as Y_dict{}
key =1
for i in indices:
    Ztst = Zhd3[i[0]:i[1]]
    Z_dict[key] = Ztst
    Ytst = Y[i[0]:i[1]]
    Y_dict[key] = Ytst
    for j in indices:
            if i != j:
                Zt = Zt + Zhd3[j[0]:j[1]]
                Z_train_dict[key] = Zt
                Yt = Yt + Y[j[0]:j[1]]
                Y_train_dict[key] = Yt
    theta_cv[key] = np.linalg.solve((np.dot(np.array(Zt).transpose(),np.array(Zt))),
                           (np.dot(np.array(Zt).transpose(),np.array(Yt))))
    key = key +1
    
#Predicting the values. Comparing the values obtained using theta_cv vs y_dict
predicthd3 ={}
for i in theta_cv:
    t_list = []
    for j in Z_dict[i]:
        t_list.append(np.dot(np.array(theta_cv[i]).transpose(),np.array(j)))
    predicthd3[i] = t_list
    
predict_trainhd3 = {}
for i in theta_cv:
    t_list = []
    for j in Z_train_dict[i]:
        t_list.append(np.dot(np.array(theta_cv[i]).transpose(),np.array(j)))
    predict_trainhd3[i] = t_list
    
MSE_testhd3 = {}
for i in Y_dict:
    sumd = 0
    for m,n in zip(predicthd3[i],Y_dict[i]):
        sumd = sumd + (m[0]-n[0])**2
    MSE_testhd3[i] = sumd/len(predicthd3[i])
    
MSE_trainhd3 = {}
for i in Y_dict:
    sumd = 0
    for m,n in zip(predict_trainhd3[i],Y_train_dict[i]):
        sumd = sumd + (m[0]-n[0])**2
    MSE_trainhd3[i] = sumd/len(predict_trainhd3[i])

In [11]:
print MSE_testhd3,'\n'
print MSE_trainhd3,'\n'
print np.mean(MSE_testhd3.values()),'\n'
print np.mean(MSE_trainhd3.values())

{1: 0.031405804119342628, 2: 0.026535042580841283, 3: 0.020007459899676287, 4: 0.026162698194302218, 5: 0.016494677126904365, 6: 0.016862490129711315, 7: 0.017209886681108209, 8: 0.016309454280746902, 9: 0.013336295191137888, 10: 0.014263644685841062} 

{1: 0.064516890137545357, 2: 0.25097719439764177, 3: 0.56338265020430223, 4: 0.96476874153682923, 5: 1.5199422573877817, 6: 2.159409709248826, 7: 2.9089205962394802, 8: 3.8075147904404991, 9: 4.8196547203199094, 10: 5.9870711313441456} 

0.019858745289 

2.30461586813


In [12]:
"cross validation procedure for 4-dimensional space"
zlen2 = len(Zhd4) #size of data given.
k_folds = 10 #number of folds to be used.
tmp_list = [] #This list will contain the floor values obtained.
indices = [] #Using the floor values we are going to determine the indices that are going to be used for cross validation.
for i in range(k_folds):
    floor = zlen2//k_folds
    tmp_list.append(floor)
ini =0 #initial value
for i in tmp_list:
    fin = ini + i #final value, final value will be used as intial value for the successive record.
    indices.append((ini,fin))
    ini = fin
#print indices

Zt = np.ndarray([]) #train
Ztst = [] #test
Yt = np.ndarray([])
Ytst = []
theta_cv = {}
Y_train_dict = {}
Z_train_dict = {}
Y_dict = {} # Contains Ytst in the form of a dictionary, for being able to handle it better with Keys.
Z_dict ={}  #Same as Y_dict{}
key =1
for i in indices:
    Ztst = Zhd4[i[0]:i[1]]
    Z_dict[key] = Ztst
    Ytst = Y[i[0]:i[1]]
    Y_dict[key] = Ytst
    for j in indices:
            if i != j:
                Zt = Zt + Zhd4[j[0]:j[1]]
                Z_train_dict[key] = Zt
                Yt = Yt + Y[j[0]:j[1]]
                Y_train_dict[key] = Yt
    theta_cv[key] = np.linalg.solve((np.dot(np.array(Zt).transpose(),np.array(Zt))),
                           (np.dot(np.array(Zt).transpose(),np.array(Yt))))
    key = key +1
    
#Predicting the values. Comparing the values obtained using theta_cv vs Z_dict
predicthd4 ={}
for i in theta_cv:
    t_list = []
    for j in Z_dict[i]:
        t_list.append(np.dot(np.array(theta_cv[i]).transpose(),np.array(j)))
    predicthd4[i] = t_list
    
predict_trainhd4 = {}
for i in theta_cv:
    t_list = []
    for j in Z_train_dict[i]:
        t_list.append(np.dot(np.array(theta_cv[i]).transpose(),np.array(j)))
    predict_trainhd4[i] = t_list
    
MSE_testhd4 = {}
for i in Y_dict:
    sumd = 0
    for m,n in zip(predicthd4[i],Y_dict[i]):
        sumd = sumd + (m[0]-n[0])**2
    MSE_testhd4[i] = sumd/len(predicthd4[i])
    
MSE_trainhd4 = {}
for i in Y_dict:
    sumd = 0
    for m,n in zip(predict_trainhd4[i],Y_train_dict[i]):
        sumd = sumd + (m[0]-n[0])**2
    MSE_trainhd4[i] = sumd/len(predict_trainhd4[i])

In [13]:
print MSE_testhd4,'\n'
print MSE_trainhd4,'\n'
print np.mean(MSE_testhd4.values()),'\n'
print np.mean(MSE_trainhd4.values())

{1: 0.025853239158815242, 2: 0.019974275303991578, 3: 0.014838810438873523, 4: 0.019504641528617354, 5: 0.012330986341131608, 6: 0.012639844471084764, 7: 0.013261161217357818, 8: 0.012718572167090219, 9: 0.011412375965425613, 10: 0.01150489008503393} 

{1: 0.064064830829775793, 2: 0.24931549918890375, 3: 0.55978094493810915, 4: 0.95795368859372931, 5: 1.5100825632821253, 6: 2.1451717994616266, 7: 2.8834414078691255, 8: 3.7691311464948276, 9: 4.7692951570839162, 10: 5.9185515593695239} 

0.0154038796677 

2.28267885971


In [14]:
'''Explicit vs Iterative using hd4'''
'''Begin iterative solution'''
len(theta_hd4) #15 features, hence we create a theta0 of 15 values.

15

In [15]:
'''for gradient descent'''
#setting a known theta0/ Assuming theta0
theta0 = np.ones(len(theta_hd4))
theta0.fill(0.5)

iteration_cnt = 300 #basically number of iterations; from 0 to 1000 we should get the correct theta value at a particular iteration.
beta = 0.00001 #Learning Rate: A value chosen such that the theta is found (theta converges)

In [16]:
for x in range(iteration_cnt):
    predict_is = [] # the for loop will iterate over Zhd4 and predict values using theta0 (the assumed theta)
    for i in Zhd4:
        predict_is.append([np.dot(theta0.transpose(),i)])
    theta_tmp = np.zeros(len(theta_hd4))
    for j in range(len(predict_is)):
        tmp = (predict_is[j][0] - Y[j][0])*Zhd4[j]
        theta_tmp = theta_tmp + tmp #implementing the iterative solution formula
    theta0 = theta0 - theta_tmp*beta

In [17]:
print theta0

[-0.02423084  0.27476301  0.07613982  0.02659217  0.09401492  0.02451174
 -0.06303124 -0.00931468 -0.03804669 -0.0218353  -0.00615162 -0.01745814
 -0.0026346  -0.01769657 -0.00545315]


In [18]:
pred = []
for j in Zhd4:
    pred.append(np.dot(np.array(theta0).transpose(),np.array(j)))

In [19]:
print pred

[-0.11718078524388445, -0.12329340884261102, -0.086688061507183251, 0.067325576530314707, 0.049718478966399561, 0.097488841959578784, -0.21183277884843402, 0.1400988274313823, -0.12080200082333267, -0.052321281978806054, -0.20372786381182642, 0.25619403609917907, -0.20556636664959602, 0.0073917970383744203, 0.084945977429404518, -0.013474134636474094, 0.018756288323588166, 0.093995476687369295, 0.1131518781547783, 0.17872979134502728, 0.16211124671173277, -0.065807457744687187, -0.060658580873957285, -0.033458252630690383, 0.050236018887979128, -0.15087936128439072, 0.12162554979996214, 0.27531115798861672, -0.51013072283227623, 0.074679314879850023, -0.11271495059281429, -0.13596542569629286, 0.28719374655089008, 0.11074111076539304, -0.13808191108255832, 0.23798610950470231, -0.10953581727023981, 0.024074919344475344, -0.15640504178552708, 0.227201655360183, -0.056348982846914829, -0.12799126946619557, 0.16231580113662553, -0.14335013763998145, 0.22017462523123393, 0.1569321362835386

In [30]:
'''predict_is1 = []
for i in range(len(theta0)):
    for j in Zhd4[i]:
        predict_is1.append(np.dot(np.array(theta0).transpose(),np.array(j)))'''

In [20]:
MSE_test_is = []
sumd = 0
for m,n in zip(pred,Y):
    sumd = sumd + (m-n)**2
MSE_test_is = sumd/len(pred)

In [21]:
print MSE_test_is

[ 0.01283283]


In [22]:
#iterative solution is better.

In [23]:
'''Gaussian Kernal Function'''
sigma = 0.11111
from scipy.spatial import distance
squared_dist = distance.cdist(Zhd4, Zhd4, 'sqeuclidean')
G = np.exp((-1/2)*(squared_dist/sigma**2))
alpha = np.linalg.solve(G,Y)

In [24]:
print alpha

[[-0.10825879]
 [-0.01290997]
 [-0.12720998]
 ..., 
 [-0.05667539]
 [-0.04032406]
 [ 0.16316383]]


In [25]:
squared_dist1 = distance.cdist(Zhd4, Zhd4, 'sqeuclidean')
G1 = np.exp((-1/2)*(squared_dist1/sigma**2))

In [26]:
predict_gauss = np.dot(alpha.transpose(),G1)

In [27]:
print predict_gauss

[[-0.10825881 -0.01290997 -0.12720998 ..., -0.05667539 -0.14304067
   0.16316383]]


In [50]:
sumd = 0
for m,n in zip(predict_gauss[0],Y):
    sumd = sumd + (m-n)**2
MSE_test_gauss = sumd/len(pred_gauss)

In [51]:
print MSE_test_gauss

[  3.15714045e-32]
